In [ ]:
#huge thanks to miu200521358 on Github for figuring out how to get OpenPose working on a Colab notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# First, change the runtime type to "GPU".
# Confirm runtime to GPU

! nvcc --version
! nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Sun Jan 17 18:16:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+

In [ ]:
# Openpose 
# Openpose version tag
ver_openpose = "v1.6.0"

In [ ]:
# If CMake is old, Openpose build fails, so download the latest version
! cmake --version

cmake version 3.12.0

CMake suite maintained and supported by Kitware (kitware.com/cmake).


In [ ]:
! wget -c "https://github.com/Kitware/CMake/releases/download/v3.17.2/cmake-3.17.2.tar.gz"
! tar xf cmake-3.17.2.tar.gz
! cd cmake-3.17.2 && ./configure && make && sudo make install

In [ ]:
# Install library

# Basic
! sudo apt-get --assume-yes update
! sudo apt-get --assume-yes install build-essential
# OpenCV
! sudo apt-get --assume-yes install libopencv-dev
# General dependencies
! sudo apt-get --assume-yes install libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler
! sudo apt-get --assume-yes install --no-install-recommends libboost-all-dev
# Remaining dependencies, 14.04
! sudo apt-get --assume-yes install libgflags-dev libgoogle-glog-dev liblmdb-dev
# Python3 libs
! sudo apt-get --assume-yes install python3-setuptools python3-dev build-essential
! sudo apt-get --assume-yes install python3-pip
! sudo -H pip3 install --upgrade numpy protobuf opencv-python
# OpenCL Generic
! sudo apt-get --assume-yes install opencl-headers ocl-icd-opencl-dev
! sudo apt-get --assume-yes install libviennacl-dev

In [ ]:
# Clone Openpose
! git clone  --depth 1 -b "$ver_openpose" https://github.com/CMU-Perceptual-Computing-Lab/openpose.git 

Cloning into 'openpose'...
remote: Enumerating objects: 657, done.
remote: Counting objects: 100% (657/657), done.
remote: Compressing objects: 100% (623/623), done.
remote: Total 657 (delta 168), reused 170 (delta 24), pack-reused 0
Receiving objects: 100% (657/657), 46.51 MiB | 39.95 MiB/s, done.
Resolving deltas: 100% (168/168), done.
Note: checking out '3d057691b219dddf264c6e412a4560ac8a12dedb'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>



In [ ]:

# Create build directory
! cd openpose && mkdir build && cd build

In [ ]:
! cd openpose && mkdir build && cd build

mkdir: cannot create directory ‘build’: File exists


In [ ]:
# https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/installation.md#cmake-command-line-configuration-ubuntu-only
# Scenario 1 - Caffe not installed and OpenCV installed using apt-get
! cd openpose/build && cmake .. 


# If you want to download the COCO model as well, execute the following command. (Please change "# ! cd" part to "! cd".)
# ! cd openpose/build && cmake .. -D DOWNLOAD_BODY_COCO_MODEL=ON
! cd openpose/build && cmake .. 
# MPIモデルのDLオプション付き（同上）
# If you want to download the MPI model as well, execute the following command.
# ! cd openpose/build && cmake .. -D DOWNLOAD_BODY_MPI_MODEL=ON

In [ ]:

# Openpose Building

! cd openpose/build && make -j`nproc`
# output
! cd openpose && mkdir output

In [ ]:
import time
time.sleep(3600)
import os
os.remove('/content/openpose/models/pose/body_25/pose_iter_584000.caffemodel')
os.remove('/content/openpose/models/face/pose_iter_116000.caffemodel')
os.remove('/content/openpose/models/hand/pose_iter_102000.caffemodel')
os.rename('/content/pose_iter_584000.caffemodel','/content/openpose/models/pose/body_25/pose_iter_584000.caffemodel')
os.rename('/content/pose_iter_116000.caffemodel','/content/openpose/models/face/pose_iter_116000.caffemodel')
os.rename('/content/pose_iter_102000.caffemodel','/content/openpose/models/hand/pose_iter_102000.caffemodel')

In [ ]:
!sudo apt-get install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.


In [ ]:
import os


counter = 0
!rm -rf "/content/30 FPS Skeleton/Right", "/content/30 FPS JSON/Right"
directories = ["/content/30 FPS Skeleton/Right", "/content/30 FPS JSON/Right"]
for dirs in directories:
	if not os.path.exists(dirs):
	    os.makedirs(dirs)



for filename in os.listdir('/content/drive/MyDrive/All Clips 30FPS (Filtered)/Right'):
  if "flipped" not in filename:
     root_path = "/content"
     file_path = "drive/MyDrive/All Clips 30FPS (Filtered)/Right/"+filename

     output_path="/content/30 FPS JSON/Right/"+filename.replace('.mp4','')
     print(output_path)
     counter=counter+1
     print("Processed Videos:")
     print(counter)
     
     ! cd openpose && ./build/examples/openpose/openpose.bin --video "$root_path/$file_path"  --render_pose 0 --display 0  --number_people_max=2  --write_json "$output_path"


In [ ]:
!zip -r "Right_Json.zip" "/content/30 FPS JSON/Right"
from google.colab import files

files.download("/content/Right_Json.zip")